# Lab 3: Streaming Codes for Low-Latency Network Communication

## Problem 1: Empirical Performance of Streaming Codes Over the Gilbert-Elliot (GE) Channel

1. Implement the function GE(...) that generates a realization of the GE channel of
length n for given channel parameters $\alpha$, $\beta$, $\epsilon_0$ , $\epsilon_1$.

In [6]:
def GE(n, alpha, beta, eps0, eps1):
    P_alpha = [1-alpha, alpha]
    X_alpha = GeneralDiscreteDistribution(P_alpha)
    
    P_beta = [1-beta, beta]
    X_beta = GeneralDiscreteDistribution(P_beta)
    
    P_eps0 = [1-eps0, eps0]
    X_eps0 = GeneralDiscreteDistribution(P_eps0)
    
    P_eps1 = [1-eps1, eps1]
    X_eps1 = GeneralDiscreteDistribution(P_eps1)
    
    sequence = []
    S = 0
    for i in range(0, n):
        if S == 1:
            sequence.append(X_eps1.get_random_element())
            if X_beta.get_random_element():
                S = 0
        elif S == 0:
            sequence.append(X_eps0.get_random_element())
            if X_alpha.get_random_element():
                S = 1
    return sequence

def expected_erasures(n, alpha, beta, eps0, eps1):
    expected_good_states = n * beta / (alpha + beta)
    expected_bad_states = n - expected_good_states
    
    return expected_good_states * eps0 + expected_bad_states * eps1

In [8]:
test_params = {"n": 100000, "alpha": 0.2, "beta": 0.1, "eps0": 1e-3, "eps1": 0.1}
# print(GE(**test_params))
print(f"observed number of erasures: {sum(GE(**test_params))}")
print(f"expected number of erasures: {round(expected_erasures(**test_params))}")

observed number of erasures: 6702
expected number of erasures: 6700


2. Implement a function MT(...) that outputs the generator matrix of an $(n, k)$ Martinian-
Trott block code based on given code parameters $T$, $B$, and $N$ , where $B < T$.

In [9]:
from itertools import chain

# You can use the following helper functions.

"""
    Finds a subset (including repeat elements) of ascending_list that sums to target_sum.
    n is only used for the recursion and should be set to len(ascending_list).
"""
def subset_sum(ascending_list, n, target_sum):
    if target_sum <= 0:
        return []
    if n ==0:
        return None
    if ascending_list[n-1] > target_sum:
        return subset_sum(ascending_list, n-1, target_sum)
    
    potential_subsol_1 = subset_sum(ascending_list, n-1, target_sum)
    if potential_subsol_1 is not None:
        return potential_subsol_1
    
    potential_subsol_2 = subset_sum(ascending_list, n-1, target_sum-ascending_list[n-1])
    if potential_subsol_2 is not None:
        return [ascending_list[n-1]] + potential_subsol_2
    
    return None



"""
    Finds a polynomial of specified degree that divides poly.
    If none exist, returns None.
"""
def find_factorpoly_of_degree(poly, degree):
    all_factors = list(factor(poly))
    
    # list of degrees of factors (factors with higher multiplicity have their degrees included multiple times)
    degree_list = list(chain(*[[f[0].degree()]*f[1] for f in all_factors]))
    
    degrees = subset_sum(degree_list, len(degree_list), degree)
    
    if degrees is None: return None
    
    included_polys = []
    for deg in degrees:
        for i in range(len(all_factors)):
            poly, multiplicity = all_factors[i]
            if poly.degree() == deg and multiplicity > 0:
                included_polys.append(poly)
                all_factors[i] = poly, multiplicity - 1
                break
                
    if not included_polys: return None
    
    return prod(included_polys)

"""
    Finds a genreator polynomial for a cyclic code with minimal field size for given n and k.
    returns the field size and a genrator polynomial.
"""
def find_cyclic_generator_poly(n, k):
    
    assert n > k
    
    # we increment the field size starting from 2 until we find a suitable polynomial
    q = 2
    while gcd(n, q) != 1: # Sage only implements codes fulfilling gcd(n, q) = 1
        q = q.next_prime_power()
    
    F.<x> = GF(q)[]
    
    # the condition that has to be met is that the polynomial divides x**n-1 - 1
    # and is of degree n-k
    gen_poly = find_factorpoly_of_degree(x**n-1, n-k)

    while gen_poly is None:
        q = q.next_prime_power()
        while gcd(n, q) != 1:
            q = q.next_prime_power()
            print(q)
        F.<x> = GF(q)[]
        gen_poly = find_factorpoly_of_degree(x**n-1, n-k)
    
    return gen_poly, q

In [10]:
def MT(T, B, N=1):
    gen_poly, q = find_cyclic_generator_poly(T, T-B)
    F.<x> = GF(q)[]
    C = codes.CyclicCode(generator_pol = gen_poly, length = T)
    gen_matrix = C.systematic_generator_matrix()
    I_B_B = identity_matrix(F,B)
    Zero_B_TminusB = zero_matrix(F, B, T-B);
    Zero_TminusB_B = zero_matrix(F, T-B, B);
    top_sub_matrix = identity_matrix(F, B).augment(Zero_B_TminusB).augment(identity_matrix(F, B))
    bottom_sub_matrix = Zero_TminusB_B.augment(gen_matrix)
    return top_sub_matrix.stack(bottom_sub_matrix)

In [11]:
MT(5,3)

[     1      0      0      0      0      1      0      0]
[     0      1      0      0      0      0      1      0]
[     0      0      1      0      0      0      0      1]
[     0      0      0      1      0      1 z2 + 1 z2 + 1]
[     0      0      0      0      1 z2 + 1 z2 + 1      1]

3.  Implement a function Domanovitz(...) that outputs the generator matrix of a random $(n,k)$ Domanovitz block code in $\mathbb{F}_{q^2}$ based on given code parameters $T,B,N$, and $q$. _Hint: Generate a random matrix whose structure is consistent with the generator matrix of a Domanovitz code, and check if the sub-matrices $G_1$ and $G_2$ satisfy the MDS property. Keep generating such random matrices until a valid generator matrix is found._

In [12]:
from collections import deque

def get_domanovitz_candidate(T, B, N, q):
    F = GF(q)
    k = T-N+1
    n = k+B
    G1_rows = []
    for _ in range(k):
        non_zero_elements = [1]
        for i in range(N-1):
            non_zero_elements.append(F.random_element())
        elements = non_zero_elements + ([0] * (k-1))
        items = deque(elements)
        items.rotate(_)
        G1_rows.append(items)
    G_1 = matrix(GF(q^2), [*G1_rows])
    G_3 = identity_matrix(GF(q^2), B-N+1)*GF(q^2).random_element()
    G_2 = matrix(GF(q^2), random_matrix(F, B-N+1))
    return G_1.augment(G_3.stack(G_2))

def Domanovitz(T, B, N, q):
    k = T-N+1
    n = k+B
    i=0
    while(i < 100):
        candidate = get_domanovitz_candidate(T,B,N,q)
        #print(candidate)
        G_1 = matrix(GF(q),candidate.submatrix(0,0,k,k+N-1))
        #print(G_1)
        test_G_1 = LinearCode(G_1).minimum_distance()
        d_G_1 = G_1.ncols()-G_1.nrows()+1
        if not (test_G_1 == d_G_1):
            continue
        G_2_nrows = k - (B-N+1)
        G_2_ncols = n - (B-N+1)
        G_2 = matrix(GF(q), candidate.submatrix(candidate.nrows()-G_2_nrows, candidate.ncols()-G_2_ncols, G_2_nrows, G_2_ncols))
        #print(G_2)
        test_G_2 = LinearCode(G_2).minimum_distance()
        d_G_2 = G_2.ncols()-G_2.nrows()+1
        if (test_G_2 == d_G_2):
            return candidate
        i += 1
    print('No matrix was found after 100 attempts')
    return None

In [13]:
Domanovitz(6, 4, 3, 11)

[       1        6        8        0        0        0 8*z2 + 2        0]
[       0        1        7        6        0        0        0 8*z2 + 2]
[       0        0        1        2        3        0       10        5]
[       0        0        0        1        6        1        8        5]

4. Plot the BEP of the codes in Table 1 as a function of $\epsilon_0\in [10^{-3},10^{-2}]$, for: 
 * i) $\alpha=   1\times 10^{-2}$, $\beta=0.3$, $\epsilon_1=1$; 
 * ii)  $\alpha= 1\times 10^{-2}$, $\beta=0.5$, $\epsilon_1=1$; and 
 * iii)  $\alpha=8\times 10^{-2}$, $\beta=0.5$, $\epsilon_1=1$. 
 
 You shall show a total of 3 graphs.

In [ ]:
from matplotlib import pyplot as plt




5. Plot the PLP of the codes in Table 1 for the same settings mentioned in the part 4. 
   
   _Hint: A source symbol $u_i$ can be decoded iff the $i$-th canonical basis vector is in the column span of the punctured generator matrix (i.e. the generator matrix with the columns corresponding to erasures removed)._

6. Interpret the results in parts 4 and 5.